In [1]:
import pandas as pd

In [3]:
#pip install clean-text[GPL]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 42.9 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171032 sha256=0da060a50cc8f8b11680f12b43afbc6deed78cd812ae68c4aa3892daa2cfea9e
  Stored in directory: /home/jovyan/.cache/pip/wheels/33/40/c1/5469c4f03ddc90bb1bd797ef3b6fe7d0d29688155ee2c05529
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [2]:
# first lets run clean_text on the 'content' column
from cleantext import clean
def clean_text(s):
    return clean(s,lower=True,                     # lowercase text
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_numbers=True,               # replace all numbers with a special token
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_number="<NUM>",
        lang="en"                   
    )



In [3]:
# split the data in chunks and run in parallel
from joblib import Parallel, delayed
from os import cpu_count

# run in parallel
def run_parallel(df, n_jobs, func):
    # call every element in the chunks in parallel
    results = Parallel(n_jobs=n_jobs)(delayed(func)(element) for element in df)
    return results

In [4]:
# clean the text
def clean_column(df):
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, clean_text)
    # replace column with cleaned text
    return results

In [9]:
#pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 92.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from nltk.tokenize import word_tokenize
# tokenize the text. run in parallel
def tokenize_column(df):
    # run the function on the data
    n_jobs = cpu_count()
    results = run_parallel(df['content'], n_jobs, word_tokenize)
    
    return results

In [6]:
from nltk.corpus import stopwords
# removing generic stopwords
def remove_stopwords(df):
    stop_words = set(stopwords.words('english'))

    # remove stopwords from the text
    def r(s):
        return [w for w in s if not w in stop_words]

    # run the function on the df
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, r)

    return results

In [7]:
# stemming the text
from nltk.stem import PorterStemmer
def stem_column(df):
    # create a stemmer
    ps = PorterStemmer()

    # stem the text
    def stem(s):
        return [ps.stem(w) for w in s]

    # run the function on the df
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, stem)
    return results

In [8]:
# remove punctiuation
import string
def remove_punctuation(df):
    # remove punctuation
    def remove_punct(s):
        return [w for w in s if w not in string.punctuation]

    # run the function on the df
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, remove_punct)
    return results

In [10]:
# loading a small part of the data for testing
df = pd.read_csv('E:/ML/DS_fake_news/news_sample.csv', index_col=0)

In [13]:
# process the data
df['content'] = clean_column(df['content'])
df['content'] = tokenize_column(df)
df['content'] = remove_stopwords(df['content'])
df['content'] = stem_column(df['content'])
df['content'] = remove_punctuation(df['content'])

In [11]:
from os import path
from os import remove
# create a file to be used for storing the data
def intialize_file(name):
    # check if file exists
    if path.exists(name):
        # if the file exists, delete it
        remove(name)
    # create the file
    with open(name, 'w') as f:
        f.write('') # write an empty string to the file to create it

In [12]:
# append data to csv file
def append_to_file(name, data):
    with open(name, 'a') as f:
        # write the data to the file
        f.write(data)

In [18]:
# create a file to store the data
tokenized_file = 'tokenized_temp.csv'
#intialize_file(tokenized_file)

# append header to the file
header = df.columns.values
append_to_file(tokenized_file, ','.join(header))
append_to_file(tokenized_file, '\n')

In [19]:
df = pd.read_csv('data/news_cleaned_2018_02_13.csv', chunksize=1000, index_col=0)

count = 0
for chunk in df:
    # process data
    chunk['content'] = clean_column(chunk['content'])
    chunk['content'] = tokenize_column(chunk)
    chunk['content'] = remove_stopwords(chunk['content'])
    chunk['content'] = stem_column(chunk['content'])
    chunk['content'] = remove_punctuation(chunk['content'])
    # append data to file
    chunk.to_csv(tokenized_file, mode='a', header=False)

    count += 1
    print(count, 'chunks processed')
    if count == 100:
        break


FileNotFoundError: [Errno 2] No such file or directory: 'D:/FakeNews_data/news.csv/news_cleaned_2018_02_13.csv'

In [54]:
# load from file
tokenized_file = 'tokenized_temp.csv'
df = pd.read_csv(tokenized_file)
# load list from string
from ast import literal_eval
df['content'] = df['content'].apply(literal_eval)

In [55]:

# count the tokens
def count(s):
    return Counter(s)

# count token frequency
from collections import Counter
def count_tokens(df):


    # run the function on the df
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, count)

    # total token frequency
    total = {}
    for list in results:
        for k,v in list.items():
            if k in total:
                total[k] += v
            else:
                total[k] = v

    return total

In [56]:
token_freq = count_tokens(df['content'])

In [57]:
# sort the tokens by frequency
token_freq = sorted(token_freq.items(), key=lambda x: x[1], reverse=True)

In [58]:
# creating a set of stopwords from the frequency list
stop_words = set()

In [59]:
# find the 80% percentile
# first finding the total number of tokens
total = 0
for token in token_freq:
    total += token[1]
# then finding the 80% percentile
percentile = int(total * 0.8)
# then finding the token that corresponds to the 80% percentile
total = 0
for token in token_freq:
    total += token[1]
    if total > percentile:
        index = token_freq.index(token)
        break
# add all tokens after the 80% percentile to the stopword list
for token in token_freq[index:]:
    stop_words.add(token[0])

In [60]:
len(stop_words)

384865

In [63]:
# remove custom stopwords from the text
def remove_costume_stopwords(df):
    # remove stopwords from the text
    def r(s):
        return [w for w in s if not w in stop_words]

    # run the function on the df with multiple threads
    n_jobs = cpu_count()
    results = run_parallel(df, n_jobs, r)

    return results


In [66]:
df['content'] = [[w for w in s if not w in stop_words] for s in df['content']]

In [67]:
# save the data
df.to_csv('ready_data.csv', index=False, header=True)

In [87]:
import pandas as pd
df = pd.read_csv('ready_data.csv')
# load list from string
from ast import literal_eval
df['content'] = df['content'].apply(literal_eval)

In [14]:
# split the data into train, test and validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['type'], test_size=0.2, random_state=42)
X_test , X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [17]:
y_test.value_counts()*8 / y_train.value_counts()

1    0.955224
0    1.090909
Name: type, dtype: float64

In [16]:
# making labels binary
y_train = y_train.apply(lambda x: 1 if x in ['fake', 'junksci', 'hate', 'clickbait',] else 0)
y_test = y_test.apply(lambda x: 1 if x in ['fake', 'junksci', 'hate', 'clickbait',] else 0)
y_val = y_val.apply(lambda x: 1 if x in ['fake', 'junksci', 'hate', 'clickbait',] else 0)

In [18]:
import numpy as np
# function which pads pandas series
def pad_series(s, max_len):
    # truncate the series if it is longer than max_len
    s = s.apply(lambda x: x[:max_len])

    # pad the series
    s = s.apply(lambda x: x + ['<pad>'] * (max_len - len(x)))

    # convert the series to np array
    s = np.array(s.tolist())
    return s

# pad the series and convert them to np arrays
X_test = pad_series(X_test, 750)
X_train = pad_series(X_train, 750)
X_val = pad_series(X_val, 750)

In [19]:
# converting the strings to integers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# fit the encoder on the training / testing / valdiation data
le.fit(np.concatenate((X_train.flatten(), X_test.flatten(), X_val.flatten())))

# transform the data
X_train = le.transform(X_train.flatten()).reshape(X_train.shape)
X_test = le.transform(X_test.flatten()).reshape(X_test.shape)
X_val = le.transform(X_val.flatten()).reshape(X_val.shape)


In [26]:
# making a baseline model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# create a dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")
# fit the classifier
dummy_clf.fit(X_train, y_train)

# predict the labels
y_pred = dummy_clf.predict(X_test)

# calculate the accuracy
accuracy_score(y_test, y_pred)


0.64

In [29]:
# making a baseline model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# create a dummy classifier
logistic_clf = LogisticRegression(max_iter=10000, solver='saga')
# fit the classifier
logistic_clf.fit(X_train, y_train)

# predict the labels
y_pred = logistic_clf.predict(X_test)

# calculate the accuracy
accuracy_score(y_test, y_pred)

0.68

In [30]:
# baseline model
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# create a dummy classifier
MLP_clf = MLPClassifier(max_iter=10000, hidden_layer_sizes=(750,1000,50))
# fit the classifier
MLP_clf.fit(X_train, y_train)

# predict the labels
y_pred = MLP_clf.predict(X_test)

# calculate the accuracy
accuracy_score(y_test, y_pred)


0.68

In [55]:
# load in the LIAR dataset
import pandas as pd
df = pd.read_csv('E:/ML/DS_fake_news/train.tsv', sep='\t', header=None, usecols=[1,2])
liar_labels = {
    'false': 1,
    'pants-fire': 1,
    'barely-true': 1,
    'mostly-true': 0,
    'true': 0,
    'half-true': 0
}
df[1] = df[1].map(liar_labels)

In [56]:
# rename the columns
df.columns = ['type', 'content']

In [57]:
df['content'] = clean_column(df['content'])
df['content'] = tokenize_column(df)
df['content'] = remove_stopwords(df['content'])
df['content'] = stem_column(df['content'])
df['content'] = remove_punctuation(df['content'])


In [58]:
# remove tokens that are not in the vectorizer
def remove_tokens(s):
    return [w for w in s if w in le.classes_]

df['content'] = df['content'].apply(remove_tokens)

In [59]:
X = pad_series(df['content'], 750)

In [60]:
# transform the data
X = le.transform(X.flatten()).reshape(X.shape)

In [61]:
y_pred = dummy_clf.predict(X)
accuracy_score(df['type'], y_pred)

0.43828125

In [62]:
y_pred = logistic_clf.predict(X)
accuracy_score(df['type'], y_pred)

0.47333984375

In [63]:
y_pred = MLP_clf.predict(X)
accuracy_score(df['type'], y_pred)

0.49658203125